### Imports

In [162]:
from tqdm import tqdm
from utils.utils import *

import networkx as nx
import numpy as np
import pandas as pd
import os
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.32 ms (started: 2023-05-22 13:05:19 -05:00)


### Parameters

In [163]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

/home/ealvarez/Project/GNN_Filter
Group:		 ['pck1^', 2]
time: 2.46 ms (started: 2023-05-22 13:05:19 -05:00)


### Load dataset

In [164]:
# Load dataset
df_process = pd.read_excel("{}/input/Edwin_proyecto2/Edwin_Set2-processed.xlsx".format(dir), header=0, sheet_name=0)
print(df_process.shape)
df_process

(460, 90)


,Set,Cluster,ID,ionMz,Label,59.0137,71.0139,74.0249,85.0296,87.0086,...,521.984,545.3994,604.0696,606.0744,611.144,664.117,665.2139,766.1076,784.1495,808.118
0,A,WT,1,0001 / zwf1^ 3.4,zwf1^,53182,641,5734,1042,1294,...,23348,6988,64597,114975,7538,4107,3095,3877,2040,3349
1,A,WT,2,0002 / zwf1^ 3.4,zwf1^,57902,1767,2889,1228,1531,...,26203,5613,75636,131864,8365,4607,4145,5541,2816,6021
2,A,WT,3,0003 / zwf1^ 3.4,zwf1^,51861,491,3540,1054,2448,...,22678,6304,64496,114834,7459,4293,3090,3279,1465,5946
3,A,WT,4,0004 / zwf1^ 3.4,zwf1^,62812,1648,6611,914,2578,...,25871,7033,76118,137734,6667,6285,4698,4961,2726,7357
4,A,WT,5,0005 / zwf1^ 3.4,zwf1^,54291,1525,3965,872,1812,...,22464,6646,67845,121045,8957,5474,4182,5072,2647,5559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,B,WT,2184,184 / zwf1-,zwf1^,181058,34300,94410,53978,17378,...,265074,142165,611985,1253220,41901,312792,13867,23979,23604,52350
456,B,WT,2185,185 / zwf1-,zwf1^,133247,35946,91914,66589,20837,...,235053,129635,575883,1157148,41314,302833,12741,21145,22037,54274
457,B,WT,2186,186 / zwf1-,zwf1^,156753,34937,90135,58742,19972,...,272005,143109,626983,1295456,44074,327254,14620,24958,22242,57179
458,B,WT,2187,187 / zwf1-,zwf1^,192830,31011,88636,48706,16375,...,234966,136943,595416,1208533,34467,317512,12937,22864,20562,54420


time: 309 ms (started: 2023-05-22 13:05:19 -05:00)


In [165]:
df_process.drop(["Set", "Cluster", "ID", "Label"], inplace=True, axis=1)
df_process

,ionMz,59.0137,71.0139,74.0249,85.0296,87.0086,88.0405,89.0245,101.0243,102.0562,...,521.984,545.3994,604.0696,606.0744,611.144,664.117,665.2139,766.1076,784.1495,808.118
0,0001 / zwf1^ 3.4,53182,641,5734,1042,1294,1940,2755,902,1260,...,23348,6988,64597,114975,7538,4107,3095,3877,2040,3349
1,0002 / zwf1^ 3.4,57902,1767,2889,1228,1531,990,2511,606,1105,...,26203,5613,75636,131864,8365,4607,4145,5541,2816,6021
2,0003 / zwf1^ 3.4,51861,491,3540,1054,2448,1928,1608,677,1154,...,22678,6304,64496,114834,7459,4293,3090,3279,1465,5946
3,0004 / zwf1^ 3.4,62812,1648,6611,914,2578,1633,2480,581,1096,...,25871,7033,76118,137734,6667,6285,4698,4961,2726,7357
4,0005 / zwf1^ 3.4,54291,1525,3965,872,1812,1348,2306,1630,803,...,22464,6646,67845,121045,8957,5474,4182,5072,2647,5559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,184 / zwf1-,181058,34300,94410,53978,17378,26847,58738,21390,40141,...,265074,142165,611985,1253220,41901,312792,13867,23979,23604,52350
456,185 / zwf1-,133247,35946,91914,66589,20837,35502,81753,22785,48479,...,235053,129635,575883,1157148,41314,302833,12741,21145,22037,54274
457,186 / zwf1-,156753,34937,90135,58742,19972,35555,73766,25427,41248,...,272005,143109,626983,1295456,44074,327254,14620,24958,22242,57179
458,187 / zwf1-,192830,31011,88636,48706,16375,23797,52303,24101,28432,...,234966,136943,595416,1208533,34467,317512,12937,22864,20562,54420


time: 17.2 ms (started: 2023-05-22 13:05:20 -05:00)


In [166]:
df_process.set_index("ionMz", inplace=True)
df_process.head()

,59.0137,71.0139,74.0249,85.0296,87.0086,88.0405,89.0245,101.0243,102.0562,109.0407,...,521.9840,545.3994,604.0696,606.0744,611.1440,664.1170,665.2139,766.1076,784.1495,808.1180
ionMz,,,,,,,,,,,,,,,,,,,,,
0001 / zwf1^ 3.4,53182,641,5734,1042,1294,1940,2755,902,1260,4804,...,23348,6988,64597,114975,7538,4107,3095,3877,2040,3349
0002 / zwf1^ 3.4,57902,1767,2889,1228,1531,990,2511,606,1105,2865,...,26203,5613,75636,131864,8365,4607,4145,5541,2816,6021
0003 / zwf1^ 3.4,51861,491,3540,1054,2448,1928,1608,677,1154,2347,...,22678,6304,64496,114834,7459,4293,3090,3279,1465,5946
0004 / zwf1^ 3.4,62812,1648,6611,914,2578,1633,2480,581,1096,4227,...,25871,7033,76118,137734,6667,6285,4698,4961,2726,7357
0005 / zwf1^ 3.4,54291,1525,3965,872,1812,1348,2306,1630,803,2882,...,22464,6646,67845,121045,8957,5474,4182,5072,2647,5559


time: 18.7 ms (started: 2023-05-22 13:05:20 -05:00)


In [167]:
df_join_raw = df_process.T
df_join_raw

ionMz,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,179 / zwf1-,180 / zwf1-,181 / zwf1-,182 / zwf1-,183 / zwf1-,184 / zwf1-,185 / zwf1-,186 / zwf1-,187 / zwf1-,188 / zwf1-
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,113929,146501,181591,185130,193451,181058,133247,156753,192830,146046
71.0139,641,1767,491,1648,1525,1562,1461,896,1076,1122,...,23820,31311,36830,35652,36922,34300,35946,34937,31011,38195
74.0249,5734,2889,3540,6611,3965,5835,5179,4241,4610,5016,...,71486,77843,98272,94198,92972,94410,91914,90135,88636,85284
85.0296,1042,1228,1054,914,872,1086,324,382,565,445,...,41599,45479,56487,55858,58914,53978,66589,58742,48706,55789
87.0086,1294,1531,2448,2578,1812,2250,1500,1614,1669,1684,...,11334,11960,17031,18786,16818,17378,20837,19972,16375,17259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,4107,4607,4293,6285,5474,6415,4188,4931,6117,6443,...,317854,337259,329269,311625,302254,312792,302833,327254,317512,259070
665.2139,3095,4145,3090,4698,4182,3481,3785,3012,3053,3870,...,12793,12478,13417,12677,11974,13867,12741,14620,12937,12665
766.1076,3877,5541,3279,4961,5072,4482,4296,3457,4989,5657,...,22267,25527,24786,25263,22627,23979,21145,24958,22864,17597
784.1495,2040,2816,1465,2726,2647,2706,2594,3299,2659,2946,...,19692,24320,22938,22576,22777,23604,22037,22242,20562,19019


time: 24.2 ms (started: 2023-05-22 13:05:20 -05:00)


In [168]:
# get WT groud
df_wt = df_join_raw.filter(regex="WT", axis=1)

print(df_wt.shape)
df_wt

(85, 230)


ionMz,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,119 / WT,120 / WT,121 / WT,122 / WT,123 / WT,124 / WT,125 / WT,126 / WT,127 / WT,128 / WT
59.0137,56523,24625,78586,76559,67449,74919,88152,99718,86001,93962,...,564307,592487,574557,576154,510624,552888,548340,493758,589787,566486
71.0139,1438,570,577,1272,1060,2181,1447,2181,1018,2682,...,46614,42273,36409,40698,45019,42277,36847,40229,42434,41677
74.0249,3904,1531,2486,2501,2557,3038,4589,4903,3301,3871,...,46339,53014,48024,46621,48806,48251,47041,43139,53010,53715
85.0296,1167,518,1260,1565,1345,660,2002,1902,1242,1842,...,63798,66069,58653,63376,66018,66352,64482,68411,73022,64475
87.0086,1646,1068,1095,1673,1596,2976,1447,2023,1016,2861,...,17958,17651,18848,17739,19534,17530,16471,14856,17663,18082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,6184,3181,8580,6189,6370,7558,6912,7220,8283,7569,...,99381,115862,111720,105290,108736,113958,101004,109009,110981,107413
665.2139,3014,2565,5077,3498,3947,3792,4221,4780,3943,5037,...,6804,8199,9240,6735,7845,8240,8055,8336,8154,7285
766.1076,4161,1987,5141,5228,4743,4405,3699,6207,4290,5554,...,2728,3742,3346,2605,3780,2001,2981,3690,5122,4044
784.1495,2455,1553,2404,3433,1996,3220,2824,4094,3115,3366,...,4808,5715,6093,4818,4699,6038,4781,4748,5513,5259


time: 18.5 ms (started: 2023-05-22 13:05:20 -05:00)


In [169]:
# get ZW groud
df_zw = df_join_raw.filter(like="zwf1^")

print(df_zw.shape)
df_zw

(85, 120)


ionMz,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0861 / zwf1^ 1.4,0862 / zwf1^ 1.4,0863 / zwf1^ 1.4,0864 / zwf1^ 1.4,0865 / zwf1^ 1.4,0866 / zwf1^ 1.4,0867 / zwf1^ 1.4,0868 / zwf1^ 1.4,0869 / zwf1^ 1.4,0870 / zwf1^ 1.4
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,134377,133636,129520,122223,133150,129628,127563,116371,114443,118647
71.0139,641,1767,491,1648,1525,1562,1461,896,1076,1122,...,1826,2021,2032,2368,2486,1838,1962,2126,2238,1541
74.0249,5734,2889,3540,6611,3965,5835,5179,4241,4610,5016,...,10884,11484,8102,8899,8554,10490,7264,6332,8344,5619
85.0296,1042,1228,1054,914,872,1086,324,382,565,445,...,1009,1706,2015,1522,2002,2012,1931,1623,1290,1562
87.0086,1294,1531,2448,2578,1812,2250,1500,1614,1669,1684,...,2781,2102,2314,2390,1518,3117,2241,1771,2458,2195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,4107,4607,4293,6285,5474,6415,4188,4931,6117,6443,...,32383,28768,29741,26118,31099,30236,28520,30316,24865,26544
665.2139,3095,4145,3090,4698,4182,3481,3785,3012,3053,3870,...,3121,3216,3339,2779,3753,3880,3325,4474,3591,2827
766.1076,3877,5541,3279,4961,5072,4482,4296,3457,4989,5657,...,7043,6511,7320,5474,5204,5873,7379,5283,6355,4407
784.1495,2040,2816,1465,2726,2647,2706,2594,3299,2659,2946,...,3673,2622,4216,4310,2780,2777,2591,2820,3410,2109


time: 22.2 ms (started: 2023-05-22 13:05:20 -05:00)


In [170]:
# get pc groud
df_pc = df_join_raw.filter(like="pck1^")

print(df_pc.shape)
df_pc

(85, 80)


ionMz,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
59.0137,33968,22565,36285,30275,34858,28558,36375,31684,25583,24293,...,59883,65860,65463,64715,52758,56120,60780,64502,62149,66849
71.0139,1496,1443,1810,678,1257,1054,1359,1724,633,492,...,2385,2623,1669,1298,2375,2198,1873,2485,2791,3162
74.0249,3792,2265,4060,3406,4116,2945,3850,3086,1624,3396,...,12879,15874,10816,12709,8690,10344,11842,14550,10949,12336
85.0296,1289,1072,1329,622,2149,1751,994,1168,1169,1198,...,3630,2871,3290,3079,2307,2160,2829,3252,2521,2340
87.0086,2829,1470,2535,981,1629,1801,1832,1584,1728,1207,...,4834,3073,3448,4773,2549,3583,3565,4005,2083,2122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,15937,12166,14447,12518,14631,11045,15289,14697,15046,15427,...,15270,16612,16990,15626,18024,16297,15362,14974,14933,18387
665.2139,4800,3541,3850,3170,4577,2327,3639,3433,4318,2868,...,4145,3679,3812,3254,3871,3197,3823,4835,3082,4443
766.1076,7941,5093,7393,5731,8116,4296,8087,6098,5020,5579,...,6320,7097,7253,6014,6647,7863,8341,9861,5991,6975
784.1495,4592,3797,4266,4448,3130,3318,3562,3451,4279,3242,...,5121,3943,4381,4225,3370,6469,3617,4152,4240,3848


time: 16.5 ms (started: 2023-05-22 13:05:20 -05:00)


In [171]:
# Logarithm

if group[0] == "WT": # Unable this line when it's run by group
  df_join_raw = df_wt.copy()
elif group[0] == "zwf1^":
  df_join_raw = df_zw.copy()
elif group[0] == "pck1^":
  df_join_raw = df_pc.copy()

df_raw_log = df_join_raw.copy()
for column in df_raw_log.columns:
  df_raw_log[column] = np.log10(df_raw_log[column])

df_raw_log

ionMz,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
59.0137,4.531070,4.353435,4.559727,4.481084,4.542302,4.455728,4.560803,4.500840,4.407951,4.385481,...,4.777304,4.818622,4.815996,4.811005,4.722288,4.749118,4.783761,4.809573,4.793434,4.825095
71.0139,3.174932,3.159266,3.257679,2.831230,3.099335,3.022841,3.133219,3.236537,2.801404,2.691965,...,3.377488,3.418798,3.222456,3.113275,3.375664,3.342028,3.272538,3.395326,3.445760,3.499962
74.0249,3.578868,3.355068,3.608526,3.532245,3.614475,3.469085,3.585461,3.489396,3.210586,3.530968,...,4.109882,4.200686,4.034067,4.104111,3.939020,4.014689,4.073425,4.162863,4.039374,4.091174
85.0296,3.110253,3.030195,3.123525,2.793790,3.332236,3.243286,2.997386,3.067443,3.067815,3.078457,...,3.559907,3.458033,3.517196,3.488410,3.363048,3.334454,3.451633,3.512151,3.401573,3.369216
87.0086,3.451633,3.167317,3.403978,2.991669,3.211921,3.255514,3.262925,3.199755,3.237544,3.081707,...,3.684307,3.487563,3.537567,3.678791,3.406370,3.554247,3.552060,3.602603,3.318689,3.326745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,4.202407,4.085148,4.159778,4.097535,4.165274,4.043166,4.184379,4.167229,4.177421,4.188281,...,4.183839,4.220422,4.230193,4.193848,4.255851,4.212108,4.186448,4.175338,4.174147,4.264511
665.2139,3.681241,3.549126,3.585461,3.501059,3.660581,3.366796,3.560982,3.535674,3.635283,3.457579,...,3.617525,3.565730,3.581153,3.512418,3.587823,3.504743,3.582404,3.684396,3.488833,3.647676
766.1076,3.899875,3.706974,3.868821,3.758230,3.909342,3.633064,3.907787,3.785187,3.700704,3.746556,...,3.800717,3.851075,3.860518,3.779163,3.822626,3.895588,3.921218,3.993921,3.777499,3.843544
784.1495,3.662002,3.579441,3.630021,3.648165,3.495544,3.520876,3.551694,3.537945,3.631342,3.510813,...,3.709355,3.595827,3.641573,3.625827,3.527630,3.810837,3.558349,3.618257,3.627366,3.585235


time: 38.5 ms (started: 2023-05-22 13:05:20 -05:00)


In [172]:
# Split graph

list_raw = []

list_raw = [df_raw_log] # only for create one graph
print(len(list_raw))
list_raw[0]

1


ionMz,0011 / pck1^ 2.4,0012 / pck1^ 2.4,0013 / pck1^ 2.4,0014 / pck1^ 2.4,0015 / pck1^ 2.4,0016 / pck1^ 2.4,0017 / pck1^ 2.4,0018 / pck1^ 2.4,0019 / pck1^ 2.4,0020 / pck1^ 2.4,...,0881 / pck1^ 2.3,0882 / pck1^ 2.3,0883 / pck1^ 2.3,0884 / pck1^ 2.3,0885 / pck1^ 2.3,0886 / pck1^ 2.3,0887 / pck1^ 2.3,0888 / pck1^ 2.3,0889 / pck1^ 2.3,0890 / pck1^ 2.3
59.0137,4.531070,4.353435,4.559727,4.481084,4.542302,4.455728,4.560803,4.500840,4.407951,4.385481,...,4.777304,4.818622,4.815996,4.811005,4.722288,4.749118,4.783761,4.809573,4.793434,4.825095
71.0139,3.174932,3.159266,3.257679,2.831230,3.099335,3.022841,3.133219,3.236537,2.801404,2.691965,...,3.377488,3.418798,3.222456,3.113275,3.375664,3.342028,3.272538,3.395326,3.445760,3.499962
74.0249,3.578868,3.355068,3.608526,3.532245,3.614475,3.469085,3.585461,3.489396,3.210586,3.530968,...,4.109882,4.200686,4.034067,4.104111,3.939020,4.014689,4.073425,4.162863,4.039374,4.091174
85.0296,3.110253,3.030195,3.123525,2.793790,3.332236,3.243286,2.997386,3.067443,3.067815,3.078457,...,3.559907,3.458033,3.517196,3.488410,3.363048,3.334454,3.451633,3.512151,3.401573,3.369216
87.0086,3.451633,3.167317,3.403978,2.991669,3.211921,3.255514,3.262925,3.199755,3.237544,3.081707,...,3.684307,3.487563,3.537567,3.678791,3.406370,3.554247,3.552060,3.602603,3.318689,3.326745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,4.202407,4.085148,4.159778,4.097535,4.165274,4.043166,4.184379,4.167229,4.177421,4.188281,...,4.183839,4.220422,4.230193,4.193848,4.255851,4.212108,4.186448,4.175338,4.174147,4.264511
665.2139,3.681241,3.549126,3.585461,3.501059,3.660581,3.366796,3.560982,3.535674,3.635283,3.457579,...,3.617525,3.565730,3.581153,3.512418,3.587823,3.504743,3.582404,3.684396,3.488833,3.647676
766.1076,3.899875,3.706974,3.868821,3.758230,3.909342,3.633064,3.907787,3.785187,3.700704,3.746556,...,3.800717,3.851075,3.860518,3.779163,3.822626,3.895588,3.921218,3.993921,3.777499,3.843544
784.1495,3.662002,3.579441,3.630021,3.648165,3.495544,3.520876,3.551694,3.537945,3.631342,3.510813,...,3.709355,3.595827,3.641573,3.625827,3.527630,3.810837,3.558349,3.618257,3.627366,3.585235


time: 34.8 ms (started: 2023-05-22 13:05:20 -05:00)


In [173]:
# Transpose

list_raw_t = []
for k in range(len(list_raw)):
  list_raw_t.append(transpose(list_raw[k]))

print(len(list_raw_t))
list_raw_t[0]

1


,59.0137,71.0139,74.0249,85.0296,87.0086,88.0405,89.0245,101.0243,102.0562,109.0407,...,521.9840,545.3994,604.0696,606.0744,611.1440,664.1170,665.2139,766.1076,784.1495,808.1180
0,4.531070,3.174932,3.578868,3.110253,3.451633,2.838849,3.292256,3.016197,2.853698,3.337260,...,4.236084,3.522705,4.638928,5.088271,3.943841,4.202407,3.681241,3.899875,3.662002,3.786325
1,4.353435,3.159266,3.355068,3.030195,3.167317,2.914343,3.289366,3.024075,2.891537,2.876218,...,4.102537,3.303196,4.519119,4.955178,3.830204,4.085148,3.549126,3.706974,3.579441,3.623766
2,4.559727,3.257679,3.608526,3.123525,3.403978,2.482874,3.478422,3.206826,2.938520,3.195900,...,4.268555,3.466719,4.671580,5.140781,3.976946,4.159778,3.585461,3.868821,3.630021,3.800648
3,4.481084,2.831230,3.532245,2.793790,2.991669,2.634477,3.319106,3.278982,2.281033,3.297323,...,4.172165,3.415641,4.556013,5.029700,3.893318,4.097535,3.501059,3.758230,3.648165,3.671728
4,4.542302,3.099335,3.614475,3.332236,3.211921,2.949878,3.301898,3.196453,3.095518,3.432007,...,4.248733,3.539954,4.616665,5.123018,4.026411,4.165274,3.660581,3.909342,3.495544,3.795115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,4.749118,3.342028,4.014689,3.334454,3.554247,3.320562,3.530456,3.420616,3.209515,3.861833,...,4.427957,3.225568,4.683371,5.152970,4.072948,4.212108,3.504743,3.895588,3.810837,3.755570
76,4.783761,3.272538,4.073425,3.451633,3.552060,3.204120,3.672652,3.352761,3.440594,3.863442,...,4.419890,3.324488,4.705462,5.185202,4.086182,4.186448,3.582404,3.921218,3.558349,3.853029
77,4.809573,3.395326,4.162863,3.512151,3.602603,3.246991,3.481012,3.352183,2.969416,3.781181,...,4.450342,3.234011,4.710836,5.160948,4.044187,4.175338,3.684396,3.993921,3.618257,3.817301
78,4.793434,3.445760,4.039374,3.401573,3.318689,3.199206,3.546543,3.313656,3.404320,3.835564,...,4.395344,3.063709,4.639646,5.140360,4.061452,4.174147,3.488833,3.777499,3.627366,3.770852


time: 30.8 ms (started: 2023-05-22 13:05:21 -05:00)


In [174]:
# Correlation matrix

list_raw_corr = []
for df in tqdm(list_raw_t):
  matrix = df.corr(method="pearson") # pearson, kendall, spearman
  list_raw_corr.append(matrix)

print(len(list_raw_corr))
list_raw_corr[0]

100%|██████████| 1/1 [00:00<00:00, 228.81it/s]

1


,59.0137,71.0139,74.0249,85.0296,87.0086,88.0405,89.0245,101.0243,102.0562,109.0407,...,521.9840,545.3994,604.0696,606.0744,611.1440,664.1170,665.2139,766.1076,784.1495,808.1180
59.0137,1.000000,0.641217,0.618279,0.582678,0.606687,0.662090,0.358931,0.594576,0.694623,0.698897,...,0.747002,0.748039,0.846407,-0.089232,0.236224,0.084141,0.599352,-0.082454,0.448117,0.727570
71.0139,0.641217,1.000000,0.685576,0.677877,0.644798,0.562639,0.628376,0.714567,0.701025,0.755392,...,0.753405,0.260025,0.703036,0.011811,0.503204,-0.109729,0.462545,0.072850,0.317905,0.576250
74.0249,0.618279,0.685576,1.000000,0.698913,0.676703,0.776970,0.371889,0.635526,0.680511,0.864379,...,0.674403,0.109487,0.640746,0.446090,0.584085,0.334399,0.368193,0.355990,0.476934,0.547769
85.0296,0.582678,0.677877,0.698913,1.000000,0.664169,0.575676,0.577428,0.526030,0.592003,0.680922,...,0.717972,0.181627,0.619403,0.046614,0.517137,-0.109884,0.348732,0.140536,0.330926,0.465871
87.0086,0.606687,0.644798,0.676703,0.664169,1.000000,0.524794,0.525221,0.593820,0.593217,0.600995,...,0.725506,0.269832,0.679318,0.153905,0.602406,-0.021800,0.247869,0.253429,0.403570,0.521802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664.1170,0.084141,-0.109729,0.334399,-0.109884,-0.021800,0.333676,-0.546636,-0.114766,0.087760,0.186074,...,-0.255242,-0.038041,0.020100,0.825390,-0.010872,1.000000,0.031954,0.526057,0.364720,0.150630
665.2139,0.599352,0.462545,0.368193,0.348732,0.247869,0.328079,0.301176,0.348552,0.571134,0.518107,...,0.539010,0.245063,0.718709,0.016708,0.367504,0.031954,1.000000,-0.168924,0.503758,0.773497
766.1076,-0.082454,0.072850,0.355990,0.140536,0.253429,0.202177,-0.031889,0.051313,0.053630,0.157187,...,0.032484,-0.296755,0.123505,0.749096,0.416512,0.526057,-0.168924,1.000000,0.289034,0.131552
784.1495,0.448117,0.317905,0.476934,0.330926,0.403570,0.352566,0.139696,0.272817,0.336041,0.395109,...,0.407867,0.037327,0.618397,0.449944,0.404858,0.364720,0.503758,0.289034,1.000000,0.649613


time: 35.3 ms (started: 2023-05-22 13:05:21 -05:00)


In [175]:
# Build graph_

threshold = 0.5
i = 1

for matrix in tqdm(list_raw_corr):
  edges = build_graph_weight(matrix, threshold)

  # save
  weighted_edges = pd.DataFrame(edges, columns=["source", "target", "weight"])
  # weighted_edges.to_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], i), index=False)
  i += 1
  print(len(edges))

  G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr=["weight"])
  graph_detail(G)

print(i - 1)
weighted_edges

  0%|          | 0/1 [00:00<?, ?it/s]

1446

100%|██████████| 1/1 [00:00<00:00, 19.21it/s]


Num. nodes: 75
Num. edges: 1446

1


,source,target,weight
0,59.0137,71.0139,0.641217
1,59.0137,74.0249,0.618279
2,59.0137,85.0296,0.582678
3,59.0137,87.0086,0.606687
4,59.0137,88.0405,0.662090
...,...,...,...
1441,611.1440,808.1180,0.541042
1442,664.1170,766.1076,0.526057
1443,665.2139,784.1495,0.503758
1444,665.2139,808.1180,0.773497


time: 63.3 ms (started: 2023-05-22 13:05:21 -05:00)
